<a href="https://colab.research.google.com/github/valebl/SSZ_Clustering/blob/main/Catalogue_declustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from matplotlib.colors import ListedColormap
from sklearn.cluster import DBSCAN
%config InlineBackend.figure_format = 'svg'

import seaborn as sns

In [ ]:
write_csv = False

filepath = "/content/drive/MyDrive/SeismicSources/CPTI15_v3.0_UTM32N.xlsx"
df = pd.read_excel(filepath,sheet_name="catalogue",usecols="C,I,L,M,Q,N",engine="openpyxl")
df = df[['X','Y','MwDef','Year','DepDef']]
#df = df.dropna(axis=0,how='any')
print(df.head(10))

# Write the dataframe object into csv file
if write_csv:
    df.to_csv ("CPTI15_v3.0.csv", header=True)


   LatDef  LonDef  MwDef  Year  DepDef
0  43.464  11.882   4.86  1005     NaN
1  41.488  13.831   5.10  1005     NaN
2  41.131  14.778   4.63  1019     NaN
3  41.131  14.778   4.63  1044     NaN
4     NaN     NaN    NaN  1046     NaN
5  45.539  10.220   5.10  1065     NaN
6     NaN     NaN    NaN  1065     NaN
7  41.129  16.869   4.86  1087     NaN
8  41.899  12.477   5.10  1091     NaN
9  41.131  14.778   4.63  1094     NaN
